In [4]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import math
import pandas as pd

# #------------------------------------------------------------------------------------------------

df = torch.load('data/cleaned.pt')
prices_df = pd.read_excel('data/raw.xlsx')
window = 30
features = ['Close', 'SPX', 'DXY', 'CPI_pct', 't', 'Fed Rate']
np_data = torch.from_numpy(df[features].to_numpy())
prices = prices_df['Close'].to_numpy()
data = torch.stack([np_data[i:i+window+1] for i in range(np_data.size(0) - window-1)])
print(data.shape)

# #------------------------------------------------------------------------------------------------

class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        self.lstm1 = nn.LSTMCell(len(features), 30)
        self.lstm2 = nn.LSTMCell(30, 30)
        self.linear = nn.Linear(30, 1)

    def forward(self, input, future = 0):
        outputs = []
        h_t = torch.zeros(input.size(0), 30, dtype=torch.double)
        c_t = torch.zeros(input.size(0), 30, dtype=torch.double)
        h_t2 = torch.zeros(input.size(0), 30, dtype=torch.double)
        c_t2 = torch.zeros(input.size(0), 30, dtype=torch.double)

        for t in range(input.size(1)):
            input_t = input[:, t, :]
            h_t, c_t = self.lstm1(input_t, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs.append(output.unsqueeze(1))
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs.append(output.unsqueeze(1))
        outputs = torch.cat(outputs, dim=1)
        return outputs
    
class CustomLoss(nn.Module):
    def __init__(self, penalty_weight=1.0, sign_penalty_weight=1.0):
        super(CustomLoss, self).__init__()
        self.mse_loss = nn.MSELoss()
        self.penalty_weight = penalty_weight
        self.sign_penalty_weight = sign_penalty_weight

    def forward(self, predictions, targets):
        # Mean Squared Error Loss
        mse = self.mse_loss(predictions, targets)
        
        # Penalty for predictions close to zero
        zero_penalty = self.penalty_weight * torch.mean(torch.exp(-torch.abs(predictions)))
        
        # Penalty for incorrect signs
        sign_penalty = self.sign_penalty_weight * torch.mean((predictions * targets < 0).float())
        
        # Total loss
        total_loss = mse + zero_penalty + sign_penalty
        return total_loss

torch.Size([8453, 31, 6])


In [5]:
def train(learning_rate, data, steps):
    print("Begin Training")
    num_train = 4500
    
    input = data[:num_train,:-1]
    target = data[:num_train,-1:,0:1]
    test_input = data[num_train+window:,:-1]
    test_target = data[num_train+window:,-1:,0:1]
    
    print('shape train',input.shape, target.shape)
    print('shape test',test_input.shape, test_target.shape)
    
    seq = Sequence()
    seq.double()
    criterion = nn.MSELoss()
    optimizer = optim.LBFGS(seq.parameters(), lr=learning_rate)

    train_loss = []
    test_loss = []
    for i in range(steps):
        print('STEP: ', i)
        def closure():
            optimizer.zero_grad()
            out = seq(input)
            last = out[:, -1:, :]
            loss = criterion(last, target)
            print('loss:', loss.item())
            loss.backward()
            train_loss.append(loss.item())
            return loss
        optimizer.step(closure)

        with torch.no_grad():
            actual = []
            predicted = []
            test_prices = prices[num_train+window:]
            test = test_input[0:1]
            
            correct_up = 0
            went_up = 0
            correct_down = 0
            went_down = 0
            last_price = test_prices[0]
            
            graph_price = []
            graph_price_predict = []
            real = []
            for x in range(test.size(1)-1):
                value = test[0,x,0].item()
                actual.append(value)
            for x2 in range(test_input.size(0)):
                # graph returns
                actual.append(test_input[x2,-1,0].item())
                pred = seq(test_input[x2:x2+1,:,:])
                last = pred[:, -1:, :].item()
                predicted.append(last)
                real.append(((test_prices[x2]-last_price)/test_prices[x2])*10)
                # print loss
                if (test_prices[x2] > last_price):
                    if last > 0:
                        correct_up += 1
                    went_up +=1
                if (test_prices[x2] < last_price):
                    if last < 0:
                        correct_down += 1
                    went_down +=1
                
                # graph price
                graph_price.append(test_prices[x2])
                if last > 0:
                    graph_price_predict.append(last_price*1.001)
                else:
                    graph_price_predict.append(last_price*0.999)
                last_price = test_prices[x2]
                
                
            print(f'up correct: {correct_up}/{went_up}')
            print(f'down correct: {correct_down}/{went_down}')
            print(f'total correct: {correct_down+correct_up}/{went_up+went_down}')

            # draw the result
            array1 = actual[0:300]
            array2 = predicted[0:300]
            x1 = np.arange(len(array1))  
            x2 = np.arange(len(array2)) + test.size(1)
            plt.figure(figsize=(10, 5))
            plt.plot(x1, array1, label='Actual', color='blue')
            plt.plot(x2, array2, label='Prediction', color='grey', linestyle='--')
            plt.plot(x2, real[0:300], label='Prediction', color='red')
            plt.savefig('predict%d.pdf'%i)
            plt.close()
            
            plt.figure(figsize=(10, 5))
            plt.plot(graph_price, label='Actual', color='blue')
            plt.plot(graph_price_predict, label='Prediction', color='grey', linestyle='--')
            plt.savefig('prices%d.pdf'%i)

    return train_loss, test_loss


In [6]:

values = []
learning_rate = [0.5, 0.5, 0.4]
hidden_sizes = [15, 35, 40]
step_size = 6

# for i in range(0,3):
train_loss, test_loss = train(
    learning_rate[0], 
    data, 
    step_size
)

values.append(test_loss)

# Plot the array
# print('Done')
# for x in range(0,len(values)):
#     plt.plot(values[x], label=batch_sizes[x])

# plt.xlabel('Step')
# plt.ylabel('Test Set Loss')
# plt.title('Loss')
# plt.legend()
# plt.savefig('loss.pdf')
# plt.close()



Begin Training
shape train torch.Size([4500, 30, 6]) torch.Size([4500, 1, 1])
shape test torch.Size([3923, 30, 6]) torch.Size([3923, 1, 1])
STEP:  0
loss: 0.0457863245222807
loss: 0.03173837569677013
loss: 0.01743621731573895
loss: 0.013893072287606063
loss: 0.013027318492099257
loss: 0.012812718895602662
loss: 0.012757581244266538
loss: 0.012741803133326915
loss: 0.01273564213155293
loss: 0.012731303533147206
loss: 0.012725815734031327
loss: 0.012716592678194338
loss: 0.012699897313679744
loss: 0.012669492785272862
loss: 0.01261511579025004
loss: 0.012521132146424906
loss: 0.012363841762149856
loss: 0.012088609565492516
loss: 0.011424065600297614
loss: 0.009833487109564108
up correct: 678/2048
down correct: 1298/1860
total correct: 2596/3908
STEP:  1
loss: 0.007884789798361411
loss: 0.006914749762615174
loss: 12.562992436000345
loss: 1.7417278952514614
loss: 0.02553187189504033
loss: 0.023860155946871235
loss: 0.017248435774075328
loss: 0.011942421623624862
loss: 0.008713798270334803


In [ ]:
example_input = torch.randn(1, 30, 7).double()   # An example input with the same shape as the real input
traced_script_module = torch.jit.trace(seq, example_input)
traced_script_module.save("traced_model.pt")

In [ ]:
arr = [1, 2, 3, 4, 5, 6, 7, 8, 9,10]
arr = np.array(arr)
window = 2
num_train=5
input = torch.from_numpy(arr[:num_train-1])
input = torch.stack([input[i:i+window] for i in range(input.size(0) - window+1)])
target = torch.from_numpy(arr[1:num_train])
target = torch.stack([target[i:i+window] for i in range(target.size(0) - window+1)])
print(input)
print(target)

test_input = torch.from_numpy(arr[num_train:-1])
test_input = torch.stack([test_input[i:i+window] for i in range(test_input.size(0) - window+1)])
test_target = torch.from_numpy(arr[num_train+1:])
test_target = torch.stack([test_target[i:i+window] for i in range(test_target.size(0) - window+1)])
print(test_input)
print(test_target)

In [12]:
print(prices[4730+15:].shape)
xx = data[4730:,-1:,0:1]
print(xx.shape)

(71,)
torch.Size([71, 1, 1])
